# StyleGAN2-ADA

참조한 링크.

- https://colab.research.google.com/github/dvschultz/ml-art-colabs/blob/master/Stylegan2_ada_Custom_Training.ipynb
- https://colab.research.google.com/github/Hephyrius/Stylegan2-Ada-Google-Colab-Starter-Notebook/blob/main/Stylegan2_Ada_Colab_Starter.ipynb

StyleGAN2-ADA 모델은 텐서플로우 1 버전에서만 작동합니다. 아래 셀을 실행해서 1 버전을 사용할 수 있도록 합니다.

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


GPU가 실행 되는지 확인 합니다.

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 15 08:26:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


구글 드라이브와 연동해서 데이터를 가져옵니다.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


깃에서 StyleGAN2-ADA을 복사해 옵니다.

In [5]:
# clone git code
%cd /content/
!git clone https://github.com/KyungRyeolBaek/stylegan2-ada.git
!mkdir downloads
!mkdir datasets

/content
Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 364, done.
remote: Total 364 (delta 0), reused 0 (delta 0), pack-reused 364
Receiving objects: 100% (364/364), 56.16 MiB | 33.67 MiB/s, done.
Resolving deltas: 100% (200/200), done.


## Dataset

구글 드라이브에 훈련 할 이미지를 zip 파일로 저장해 놓습니다.

In [6]:
dataset_name = "best_images"

노트북으로 데이터셋 압축파일을 불러와서 압축을 풉니다. 왼쪽 Files에서 확인할 수 있습니다.시간이 조금 소요 됩니다.

In [7]:
import zipfile

path = "/content/drive/MyDrive/content/datasets/best_images/" # 압축 파일이 저장되어 있는 파일의 경로
local_path = "/content/best_images/" # 이미지 저장 할 경로
file_name = path + dataset_name + ".zip"
with zipfile.ZipFile(file_name, 'r') as zip:
   print('파일 압축 해제 중...') 
   zip.extractall(local_path)
   print('끝!')

파일 압축 해제 중...
끝!


불러온 이미지 데이터셋들을 tensorflow에서 사용할 수 있는 데이터셋 포멧인 tfrecords 형태로 바꿔줍니다. 데이터셋의 크기에 소요되는 시간이 다를 수 있습니다.

In [9]:
%cd /content/

# 이미지 경로 설정
dataset_path = "/content/best_images/best_images" # 저장된 이미지가 있는 폴더

# 빈 파일을 따로 생성 할 필요 없이 자동으로 생성 됩니다.
%cd /content/stylegan2-ada
!python dataset_tool.py create_from_images /content/datasets/{dataset_name} {dataset_path} # 이미지 처리 후 저장 할 곳, 저장된 이미지가 있는 경로

/content
/content/stylegan2-ada
Loading images from "/content/best_images/best_images"
Creating dataset "/content/datasets/best_images"
dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 159 images.


## Training

아래 파라미터에 대한 설명을 보고 수정 가능 합니다.

In [10]:
%cd /content/stylegan2-ada
!python train.py --help

/content/stylegan2-ada
usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with L

아래 셀이 훈련 시작 입니다. 

## 훈련 중에는 colab이 꺼질 시 처음부터 다시 실행 해야 합니다.

pkl로 학습을 진행하고 싶은 경우는 resum_from의 경로를 pkl 파일로 변경 해주면 됩니다.

In [ ]:
# output directory
output_dir = '/content/drive/MyDrive/Colab\ Notebooks/results/' + dataset_name + "/"

# config
config = "auto"

# gamma
gamma = 1

#how often should the model generate samples and a .pkl file
snapshot_count = 1

#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False

#metrics? 
metric_list = None
#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
resume_from = "ffhq1024"

# make sure there is no space in the resume path. if there is any, use a backslash character to escape
# resume_from = '/content/drive/MyDrive/Colab\ Notebooks/results/train787/00003-train787-mirror-stylegan2-gamma1-resumeffhq1024/network-snapshot-000143.pkl'

#don't edit this unless you know what you're doing :)
!python train.py --outdir={output_dir} \
                 --cfg={config} \
                 --snap={snapshot_count} \
                 --data=/content/datasets/{dataset_name} \
                 --mirror={mirrored} --mirrory={mirroredY} \
                 --gamma={gamma} \
                 --metrics={metric_list} \
                 --resume={resume_from}

tcmalloc: large alloc 4294967296 bytes == 0x55e487686000 @  0x7fcaa7ae3001 0x7fcaa4ce654f 0x7fcaa4d36b58 0x7fcaa4d3ab17 0x7fcaa4dd9203 0x55e4806d1544 0x55e4806d1240 0x55e480745627 0x55e48073fced 0x55e4806d348c 0x55e480714159 0x55e4807110a4 0x55e4806d1d49 0x55e48074594f 0x55e48073f9ee 0x55e480611e2b 0x55e480741fe4 0x55e48073f9ee 0x55e480611e2b 0x55e480741fe4 0x55e48073fced 0x55e480611e2b 0x55e480741fe4 0x55e4806d2afa 0x55e480740915 0x55e48073f9ee 0x55e48073f6f3 0x55e4808094c2 0x55e48080983d 0x55e4808096e6 0x55e4807e1163
tcmalloc: large alloc 4294967296 bytes == 0x55e587686000 @  0x7fcaa7ae11e7 0x7fcaa4ce646e 0x7fcaa4d36c7b 0x7fcaa4d3735f 0x7fcaa4dd9103 0x55e4806d1544 0x55e4806d1240 0x55e480745627 0x55e48073f9ee 0x55e4806d2bda 0x55e480741737 0x55e48073f9ee 0x55e4806d2bda 0x55e480741737 0x55e48073f9ee 0x55e4806d2bda 0x55e480741737 0x55e4806d2afa 0x55e480740915 0x55e48073f9ee 0x55e4806d2bda 0x55e480744d00 0x55e48073f9ee 0x55e4806d2bda 0x55e480741737 0x55e48073fced 0x55e4806d348c 0x55e48071

In [7]:
from PIL import Image

img = Image.open('/content/drive/MyDrive/Colab Notebooks/results/best_images/00000-best_images-mirror-auto1-gamma1-resumeffhq1024/fakes000060.jpg')

%cd /content/drive/MyDrive/Colab Notebooks/results
%mkdir crop_images

for n in range(4):
  imgNum  = 1 + (7*n)
  x, y = 0, (0 + 1024*n)
  w, h = 1024, (1024 + 1024*n)
  for i in range(1,8):
    try:
        area = (x,y,w,h)
        cropped = img.crop(area)
        cropped.save("/content/drive/MyDrive/Colab Notebooks/results/crop_images/Test_{}.png".format(imgNum),'PNG')
        imgNum += 1
        x += 1024
        w += 1024
    except:
      continue

/content/drive/MyDrive/Colab Notebooks/results
